In [1]:
import pandas as pd
import numpy as np
import copy
import sys
import snowflake.connector as connector
import matplotlib.pyplot as plt

In [13]:
st =pd.DataFrame(data=[],columns=['letter','number','dash'])

In [17]:
st.append({'letter':1,'number':2,'dash':3},ignore_index=True)

,letter,number,dash
0,1,2,3


In [19]:
help(st.append)

Help on method append in module pandas.core.frame:

append(other, ignore_index=False, verify_integrity=False, sort=False) -> 'DataFrame' method of pandas.core.frame.DataFrame instance
    Append rows of `other` to the end of caller, returning a new object.
    
    Columns in `other` that are not in the caller are added as new columns.
    
    Parameters
    ----------
    other : DataFrame or Series/dict-like object, or list of these
        The data to append.
    ignore_index : bool, default False
        If True, the resulting axis will be labeled 0, 1, …, n - 1.
    verify_integrity : bool, default False
        If True, raise ValueError on creating index with duplicates.
    sort : bool, default False
        Sort columns if the columns of `self` and `other` are not aligned.
    
        .. versionadded:: 0.23.0
        .. versionchanged:: 1.0.0
    
            Changed to not sort by default.
    
    Returns
    -------
    DataFrame
    
    See Also
    --------
    concat :

In [15]:
st0 = pd.DataFrame(data=[[1],[2],[3]],columns = ['letter','number','dash'])

ValueError: 3 columns passed, passed data had 1 columns

In [5]:
st.append([[1],[3],[4]],column= ['letter','number','dash'] )

TypeError: append() got an unexpected keyword argument 'column'

In [85]:
st.append({'letter':1,'number':4,'dash':3},ignore_index=True)

,letter,number,dash
0,0,0,0
1,1,4,3


In [2]:
def amountLetterNumber(data):
    #st =pd.DataFrame(data=[0,0,0],columns=['letter','number','dash'])
    st = set()
    for PN in data:
        lr_num = 0
        dt_num =0
        dash_num =0
        if len(PN)>7 and len(PN)<18:
            for i in PN:
                if i.isdigit():
                    dt_num+=1
                if i.isalpha():
                    lr_num+=1
                if i=='-':
                    dash_num+=1
            if dt_num>1 and lr_numb <3:
                b =(lr_num,dt_num,dash_num)
                st.add(b)
    return st

In [18]:
PN = pd.read_csv('data1/PN.txt',sep='delimiter', header=None,engine='python')

In [154]:
df3 = pd.read_excel('data1/2019_2023PO.xlsx')

In [ ]:
# retrieve data from snowflake
my_query = """
select 
a.ebeln as PO,
b.ebelp as PO_item,
ltrim(b.matnr,'0')as PN,
e.description,
c.name1 as Vendor_name,
c.BRSCH as Industry_key,
c.land1,
b.netpr as purchased_price,
b.menge as PO_qty,
a.aedat as po_date,
d.eindt as delivery_date,
a.waers as original_currency,
case when a.waers = 'USD'
          then b.netpr
          else round(b.netpr*m.exchange_rate,2)
end as unit_price_usd,
c.zzcatman as Category_manager,
f.eknam as Buyer_name,
c.zzcatego as category,
zzsubcat as SCL,
b.werks
from (select ebeln,waers,bstyp,ekgrp,lifnr,aedat, concat(waers::text,substring(aedat::text,1,6)) as uni_key from rpl_sap.ekko) as a 
left join rpl_sap.t024 as f on a.ekgrp = f.ekgrp
left join rpl_sap.ekpo as b on a.ebeln = b.ebeln
left join rpl_sap.eket as d on b.ebeln = d.ebeln and b.ebelp = d.ebelp
left join rpl_sap.lfa1 as c on a.lifnr = c.lifnr
left join core.material as e on b.matnr = e.material
left join (SELECT from_currency, to_currency, exchange_rate, date_text,concat(from_currency,date_text) as uni_key
   FROM ( SELECT c.fcurr AS from_currency,
                 c.tcurr AS to_currency, 
                CASE
                    WHEN c.ffact = 0::numeric THEN 
                    CASE
                        WHEN c.ukurs < 0::numeric THEN power(c.ukurs, (-1)::numeric) * (-1)::numeric
                        ELSE c.ukurs
                    END
                    ELSE 
                    CASE
                        WHEN c.ukurs < 0::numeric THEN power(c.ukurs, (-1)::numeric) * (-1)::numeric * c.tfact / c.ffact
                        ELSE c.ukurs * c.tfact / c.ffact
                    END
                END AS exchange_rate,
         "substring"((99999999 - c.gdatu::bigint)::text, 1, 6) AS date_text,
         row_number() OVER(
         PARTITION BY "substring"(c.gdatu::text, 1, 6), c.fcurr, c.tcurr
          ORDER BY c.gdatu) AS ranking
           FROM rpl_sap_bi.tcurr as c
          WHERE c.kurst::text = 'M'::text AND c.tcurr::text = 'USD'::text) as c
  WHERE c.ranking = 1
  and date_text >='201201'
  order by date_text
 ) as m on m.uni_key = a.uni_key 
where b.loekz !='L'
and a.bstyp = 'F'
and a.aedat >='20220101'
and purchased_price >0.01
"""
with connector.connect( 
    user='ethan.zhu@technipfmc.com', # Required. Replace with your email 
    authenticator="externalbrowser", # Required. 
    account='technipfmc-data', # Required. 
    database="idsprod", # Optional 
    schema="public", # Optional. Replace with the schema you will be working on 
    role="reporting", # Optional. Replace with the role you will be working with 
    warehouse="reporting_wh", # Optional. Replace with the warehouse you will be working with 
    client_store_temporary_credential=True, # Only if installing secure-local-storage to avoid reopening tabs
    ) as conn: 
    cursor = conn.cursor() 
    cursor.execute(my_query) 
    df = cursor.fetch_pandas_all() # To return a dataframe
df.shape,df.head(2)

In [31]:
def repeat(data):
    pn = set(data.index)
    b =set((str(2019),str(2020),str(2021),str(2022)))
    unique=[]
    for i in pn:
        m = data.loc[i]
        if b.issubset(set((m))):
            unique.append(i)
    return pd.Series(unique)

In [65]:
with pd.ExcelWriter('data1/FastenerSpendSummary1.xlsx') as writer:
    #repeatPNvsallPN.to_excel(write,sheet_name='repeatPNvsallPN',index=False)
    #repeatPNPOVSallPO.to_excel(writer, sheet_name = 'repeatPNPOVSallPO',index =False)
    repeatPN.to_excel(writer, sheet_name = 'RepeatPN19-22')#done